In [113]:
import math
import numpy as np
import pandas as pd
import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings('ignore')

In [16]:
data=pd.read_csv("/Users/yomiyama/document/Rproject/choukashibou/data/original-data/suppl3-Aug.csv",
                 encoding="SHIFTJIS")
data.head()

,prefecture_id,prefectureJP,prefectureEN,week_ending_date,補正あり死亡数,補正なし死亡数
0,1,北海道,Hokkaido,10jan2010,1082,1082
1,1,北海道,Hokkaido,17jan2010,1071,1071
2,1,北海道,Hokkaido,24jan2010,1041,1041
3,1,北海道,Hokkaido,31jan2010,1115,1115
4,1,北海道,Hokkaido,07feb2010,1089,1089


In [74]:
data['death_week']=pd.to_datetime(data['week_ending_date']).dt.strftime("%V").astype(float).astype(int)
data['week_ending_date']=pd.to_datetime(data['week_ending_date'])
data.head()
print(data)

       prefecture_id prefectureJP prefectureEN week_ending_date  補正あり死亡数  \
0                  1          北海道     Hokkaido       2010-01-10     1082   
1                  1          北海道     Hokkaido       2010-01-17     1071   
2                  1          北海道     Hokkaido       2010-01-24     1041   
3                  1          北海道     Hokkaido       2010-01-31     1115   
4                  1          北海道     Hokkaido       2010-02-07     1089   
...              ...          ...          ...              ...      ...   
25516             47           沖縄      Okinawa       2020-05-03      231   
25517             47           沖縄      Okinawa       2020-05-10      247   
25518             47           沖縄      Okinawa       2020-05-17      214   
25519             47           沖縄      Okinawa       2020-05-24      233   
25520             47           沖縄      Okinawa       2020-05-31      240   

       補正なし死亡数  death_week  
0         1082           1  
1         1071           2  


In [61]:
dw=data['death_week'].astype(float)

In [77]:
weeks=float(365.25/7)
data['sin52']=np.sin(2*np.pi/weeks*data['death_week'])
data['cos52']=np.cos(2*np.pi/weeks*data['death_week'])
data['sin26']=np.sin(4*np.pi/weeks*data['death_week'])
data['cos26']=np.cos(4*np.pi/weeks*data['death_week'])
data.head()

,prefecture_id,prefectureJP,prefectureEN,week_ending_date,補正あり死亡数,補正なし死亡数,death_week,sin52,cos52,sin26,cos26
0,1,北海道,Hokkaido,2010-01-10,1082,1082,1,0.120126,0.992759,0.238513,0.971139
1,1,北海道,Hokkaido,2010-01-17,1071,1071,2,0.238513,0.971139,0.463258,0.886224
2,1,北海道,Hokkaido,2010-01-24,1041,1041,3,0.353445,0.935455,0.661263,0.750154
3,1,北海道,Hokkaido,2010-01-31,1115,1115,4,0.463258,0.886224,0.821100,0.570784
4,1,北海道,Hokkaido,2010-02-07,1089,1089,5,0.566362,0.824157,0.933542,0.358468


In [82]:
hokkaido=data[data.prefecture_id==1]
hokkaido.shape
hokkaido.head

<bound method NDFrame.head of      prefecture_id prefectureJP prefectureEN week_ending_date  補正あり死亡数  \
0                1          北海道     Hokkaido       2010-01-10     1082   
1                1          北海道     Hokkaido       2010-01-17     1071   
2                1          北海道     Hokkaido       2010-01-24     1041   
3                1          北海道     Hokkaido       2010-01-31     1115   
4                1          北海道     Hokkaido       2010-02-07     1089   
..             ...          ...          ...              ...      ...   
538              1          北海道     Hokkaido       2020-05-03     1263   
539              1          北海道     Hokkaido       2020-05-10     1244   
540              1          北海道     Hokkaido       2020-05-17     1177   
541              1          北海道     Hokkaido       2020-05-24     1199   
542              1          北海道     Hokkaido       2020-05-31     1147   

     補正なし死亡数  death_week     sin52     cos52     sin26     cos26  
0       1082  

In [114]:
y=hokkaido[['補正あり死亡数']]
y_trans=hokkaido.iloc[0:525,4]
x=hokkaido[['death_week','sin52','cos52','sin26','cos26']]
x_trans_1=hokkaido.iloc[0:525,]
x_trans_2=x_trans_1[['death_week','sin52','cos52','sin26','cos26']]
print(x_trans_1)
link = sm.families.links.identity
family=sm.families.Poisson(link=link)

# model
model = sm.GLM(y_trans, x_trans_2, family=family)
results = model.fit()
print(results.summary())

     prefecture_id prefectureJP prefectureEN week_ending_date  補正あり死亡数  \
0                1          北海道     Hokkaido       2010-01-10     1082   
1                1          北海道     Hokkaido       2010-01-17     1071   
2                1          北海道     Hokkaido       2010-01-24     1041   
3                1          北海道     Hokkaido       2010-01-31     1115   
4                1          北海道     Hokkaido       2010-02-07     1089   
..             ...          ...          ...              ...      ...   
520              1          北海道     Hokkaido       2019-12-29     1282   
521              1          北海道     Hokkaido       2020-01-05     1425   
522              1          北海道     Hokkaido       2020-01-12     1385   
523              1          北海道     Hokkaido       2020-01-19     1336   
524              1          北海道     Hokkaido       2020-01-26     1301   

     補正なし死亡数  death_week     sin52     cos52     sin26     cos26  
0       1082           1  0.120126  0.992759

In [112]:
y_pred = results.predict(x)
print(y_pred)                         

0       500.222782
1       699.198202
2       878.480560
3      1033.643194
4      1161.376643
          ...     
538     930.322285
539     902.637906
540     889.729255
541     891.467326
542     906.655399
Length: 543, dtype: float64


In [ ]:
y = df['tip']          # 目的変数：y
x = df['total_bill']   # 説明変数：x（線形予測子：wx）
link = sm.families.links.identity()  # リンク関数：恒等リンク関数
family = sm.families.Gaussian(link)  # 確率分布：ガウス分布

In [42]:
import calendar
import datetime
datetime.date(2010, 6, 16).strftime("%V")

'24'

In [25]:
string_date_1 = '2019/08/02'
dt=datetime.datetime.strptime(string_date_1, '%Y/%m/%d') # => datetime.datetime(2019, 8, 2, 0, 0)
dt1=datetime.date(dt.year, dt.month, dt.day) # => datetime.date(2019, 8, 2)
print(dt1)

2019-08-02


In [ ]:
weeks=365.25/7
sin52=math.sin(2*math.pi/weeks*death_week)